Given a positive integer n, return the number of the integers in the range [0, n] whose binary representations do not contain consecutive ones.

 

Example 1:

Input: n = 5
Output: 5
Explanation:
Here are the non-negative integers <= 5 with their corresponding binary representations:
0 : 0
1 : 1
2 : 10
3 : 11
4 : 100
5 : 101
Among them, only integer 3 disobeys the rule (two consecutive ones) and the other 5 satisfy the rule. 
Example 2:

Input: n = 1
Output: 2
Example 3:

Input: n = 2
Output: 3
 

Constraints:

1 <= n <= 109

In [ ]:
class Solution:
    def findIntegers(self, n: int) -> int:
        # Convert n to binary and store bits (MSB first)
        bits = []
        temp = n
        while temp > 0:
            bits.append(temp & 1)
            temp >>= 1
        bits.reverse()  # MSB first
        
        def dfs(pos, prev_bit, is_limit):
            # Base case: processed all bit positions
            if pos == len(bits):
                return 1
            
            # Determine the maximum bit we can place at this position
            max_bit = bits[pos] if is_limit else 1
            
            count = 0
            for bit in range(max_bit + 1):
                # Skip if placing 1 after 1 (consecutive ones)
                if bit == 1 and prev_bit == 1:
                    continue
                
                # Update limit for next recursion
                new_limit = is_limit and (bit == bits[pos])
                
                count += dfs(pos + 1, bit, new_limit)
            
            return count
        
        return dfs(0, 0, True)
    
# Plain recursion: O(2^(log n)) 
# Why:

# At each bit position: 2 choices (place 0 or 1)
# Total bit positions: log n
# Worst case: 2^(log n) = n recursive calls 

# sc - O(log n)

In [ ]:
# Limited states are specific to n's bit pattern
# Non-limited states are reusable across different n values
# We only cache the reusable computations

class Solution:
    def findIntegers(self, n: int) -> int:
        # Convert n to binary and store bits (MSB first)
        bits = []
        temp = n
        while temp > 0:
            bits.append(temp & 1)
            temp >>= 1
        bits.reverse()  # MSB first
        
        memo = {}
        
        def dfs(pos, prev_bit, is_limit):
            # Base case: processed all bit positions
            if pos == len(bits):
                return 1
            
            # Check memo (only cache when is_limit is False)
            if not is_limit and (pos, prev_bit) in memo:
                return memo[(pos, prev_bit)]
            
            # Determine the maximum bit we can place at this position
            max_bit = bits[pos] if is_limit else 1
            
            count = 0
            for bit in range(max_bit + 1):
                # Skip if placing 1 after 1 (consecutive ones)
                if bit == 1 and prev_bit == 1:
                    continue
                
                # Update limit for next recursion
                new_limit = is_limit and (bit == bits[pos])
                
                count += dfs(pos + 1, bit, new_limit)
            
            # Cache result (only when is_limit is False)
            if not is_limit:
                memo[(pos, prev_bit)] = count
            
            return count
        
        return dfs(0, 0, True)


# tc - O(log n)
# sc - O(log n)

In [ ]:
class Solution:
    def findIntegers(self, n: int) -> int:
        # Convert n to binary (MSB first)
        bits = []
        temp = n
        while temp > 0:
            bits.append(temp & 1)
            temp >>= 1
        bits.reverse()
        
        if not bits:
            return 1
        
        num_bits = len(bits)
        
        # dp[pos][prev_bit][is_limit] = count of valid numbers
        # pos: current position (0 to num_bits-1)
        # prev_bit: previous bit (0 or 1)
        # is_limit: whether we're limited by n's bits (0 or 1)
        dp = [[[0 for _ in range(2)] for _ in range(2)] for _ in range(num_bits + 1)]
        
        # Base case: when we've placed all bits
        dp[num_bits][0][0] = dp[num_bits][0][1] = dp[num_bits][1][0] = dp[num_bits][1][1] = 1
        
        # Fill table backwards (from last position to first)
        for pos in range(num_bits - 1, -1, -1):
            for prev_bit in range(2):
                for is_limit in range(2):
                    count = 0
                    
                    # Determine max bit we can place
                    max_bit = bits[pos] if is_limit else 1
                    
                    for bit in range(max_bit + 1):
                        # Skip consecutive ones
                        if bit == 1 and prev_bit == 1:
                            continue
                        
                        # Calculate new limit
                        new_limit = 1 if (is_limit and bit == bits[pos]) else 0
                        
                        count += dp[pos + 1][bit][new_limit]
                    
                    dp[pos][prev_bit][is_limit] = count
        
        # Start with pos=0, prev_bit=0 (no previous bit), is_limit=True
        return dp[0][0][1]